# Rosbag Parser
02/11/2022
Gregory Burgess

In folder where rosbags are stored, need another folder called '/path_to_rosbags/rosbag_output/'

---
## Import Libraries

In [1]:
import bagpy
import pandas as pd
import seaborn as sea
import matplotlib.pyplot as plt
import numpy as np

from bagpy import bagreader

---
## Import Rosbags
### Insert filepath to Rosbags


In [5]:
#filepath = '/home/gburgess/dvl-nav/data/field_data/buzz_bay/dec_15/rosbags/'
filepath = '/home/gburgess/dvl-nav/data/field_data/puerto_rico/rosbags/mar6/'
# Grab all files ending with .bag in filepath
bags_raw = [bag for bag in os.listdir(path=filepath) if bag.endswith('.bag')]
# Sort rosbag files in chronological order
bags_raw.sort()

## Select Which Topics to Sort out

In [150]:
# Can look at an individual bag
# b=bagreader(filepath+bags_raw[3])
# b.topic_table

In [6]:
topics = [
    '/devices/spartonm2/ahrs',
    '/devices/dvl/instrument/raw',
    '/devices/dvl/pd0',
    '/devices/dvl/ranges',
    '/devices/dvl/dvl',
    '/extctl/sensors/m_depth',
    '/extctl/sensors/m_altitude',
    '/extctl/sensors/m_altimeter_status',
    '/extctl/sensors/m_dist_to_wpt',
    '/extctl/sensors/m_gps_lat',
    '/extctl/sensors/m_gps_lon',
    '/extctl/sensors/m_lat',
    '/extctl/sensors/m_lon',
    '/extctl/sensors/m_pitch',
    '/extctl/sensors/m_present_time',
    '/extctl/sensors/m_roll',
    '/extctl/sensors/m_water_depth',
    '/extctl/sensors/m_heading',
    '/tritech_micron_beam',
    '/tritech_micron_data',
    '/ctd/data',
    '/battery_pack/data'
]

### Parse Bags

In [7]:
# Initialize Dictionary to hold filepaths to generated CSVs
csv_dic = {}
for t in topics:
    csv_dic[t] = []

# Parse out Data by topic and generate set of names
for bag in bags_raw:
    b=bagreader(filepath+bag)
    for t in topics:
        data = b.message_by_topic(t)
        if type(data) == str:
            csv_dic[t].append(data)

[INFO]  Successfully created the data folder /home/gburgess/dvl-nav/data/field_data/puerto_rico/rosbags/mar6/2022-03-06-00-22-18_0.
No data on the topic:/extctl/sensors/m_depth
No data on the topic:/extctl/sensors/m_altitude
No data on the topic:/extctl/sensors/m_altimeter_status
No data on the topic:/extctl/sensors/m_dist_to_wpt
No data on the topic:/extctl/sensors/m_gps_lat
No data on the topic:/extctl/sensors/m_gps_lon
No data on the topic:/extctl/sensors/m_lat
No data on the topic:/extctl/sensors/m_lon
No data on the topic:/extctl/sensors/m_pitch
No data on the topic:/extctl/sensors/m_present_time
No data on the topic:/extctl/sensors/m_roll
No data on the topic:/extctl/sensors/m_water_depth
No data on the topic:/ctd/data
[INFO]  Successfully created the data folder /home/gburgess/dvl-nav/data/field_data/puerto_rico/rosbags/mar6/2022-03-06-00-40-21_0.
No data on the topic:/extctl/sensors/m_depth
No data on the topic:/extctl/sensors/m_altitude
No data on the topic:/extctl/sensors/m_a

In [8]:
names = []
for key in csv_dic:
    print(key)
    names.append(key.replace('/','-')[1:])


/devices/spartonm2/ahrs
/devices/dvl/instrument/raw
/devices/dvl/pd0
/devices/dvl/ranges
/devices/dvl/dvl
/extctl/sensors/m_depth
/extctl/sensors/m_altitude
/extctl/sensors/m_altimeter_status
/extctl/sensors/m_dist_to_wpt
/extctl/sensors/m_gps_lat
/extctl/sensors/m_gps_lon
/extctl/sensors/m_lat
/extctl/sensors/m_lon
/extctl/sensors/m_pitch
/extctl/sensors/m_present_time
/extctl/sensors/m_roll
/extctl/sensors/m_water_depth
/tritech_micron_beam
/tritech_micron_data
/ctd/data
/battery_pack/data


In [9]:
# Concatenate all csvs into one dataframe per topic
final_dic = {}
n_count=0
for key in csv_dic:
    count = 0
    temp_dic = {}
    for csv in csv_dic[key]:
        temp_dic[key+str(count)] = (pd.read_csv(csv, on_bad_lines='skip'))
        count += 1
    temp_df = pd.concat(temp_dic)
    new_df = temp_df.reset_index()
    new_df = new_df.iloc[:, 2:]
    final_dic[names[n_count]] = new_df
    n_count += 1

In [10]:
# Save final CSVs to output folder
for key in final_dic:
    final_dic[key].to_csv(filepath+'rosbag_output/'+str(key)+'.csv')